In [1]:
from keras.models import Model
from keras.applications import vgg16
from keras.preprocessing.image import load_img, img_to_array
import keras

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

Using TensorFlow backend.


In [2]:
# get image filenames for training and test sets
import glob

files = glob.glob('web_images/*')
img_files = []
for fn in files:
    img_files.append(fn)

len(img_files), img_files

(12,
 ['web_images/gulupa-100x100.jpg',
  'web_images/pears.jpg',
  'web_images/strawberries1.jpg',
  'web_images/E49BE62E-6943-E411-B834-22000AF88B16.jpg',
  'web_images/F8CFCFB3-D645-E411-B834-22000AF88B16.jpg',
  'web_images/A2657DDD-4643-E411-B834-22000AF88B16.jpg',
  'web_images/Fruit-snacking-Future-trends-and-challenges_small.jpg',
  'web_images/Boone+Olive+Oil+CO.+Whole+Fruit+Lemon+Olive+Oil.jpg',
  'web_images/apple-100px.jpg',
  'web_images/home-basics-fruit-baskets-fb00459-c3_100.jpg',
  'web_images/4frk13.jpg',
  'web_images/pluot_splash.jpg'])

In [3]:
# load up the VGG_16 model
vgg = vgg16.VGG16(include_top=False, weights='imagenet', 
                                     input_shape=(100,100,3))

output = vgg.layers[-1].output
output = keras.layers.Flatten()(output)
vgg_model = Model(vgg.input, output)

vgg_model.trainable = False
for layer in vgg_model.layers:
    layer.trainable = False
    
pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])   

Instructions for updating:
Colocations handled automatically by placer.


,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at 0xb3f8854a8>,input_1,False
1,<keras.layers.convolutional.Conv2D object at 0x10d7b4dd8>,block1_conv1,False
2,<keras.layers.convolutional.Conv2D object at 0xb3f81a198>,block1_conv2,False
3,<keras.layers.pooling.MaxPooling2D object at 0xb3f900cc0>,block1_pool,False
4,<keras.layers.convolutional.Conv2D object at 0xb3f900da0>,block2_conv1,False
5,<keras.layers.convolutional.Conv2D object at 0xb3f9474a8>,block2_conv2,False
6,<keras.layers.pooling.MaxPooling2D object at 0xb3f9632b0>,block2_pool,False
7,<keras.layers.convolutional.Conv2D object at 0xb3f9638d0>,block3_conv1,False
8,<keras.layers.convolutional.Conv2D object at 0xb3f9946d8>,block3_conv2,False
9,<keras.layers.convolutional.Conv2D object at 0xb3f9b04a8>,block3_conv3,False


In [4]:
# load training images and extract features

IMG_DIM = (100,100)
features, labels = [], []
for idx, img in enumerate(img_files):
    img_array = np.array([img_to_array(load_img(img,target_size=IMG_DIM))])
    # scale pixels between (0,1)
    img_array = img_array.astype('float32') / 255
    features.append( vgg_model.predict(img_array, verbose=0)[0] )
    labels.append( img )
    if idx % 1000 == 0: print(idx)
        
features_df = pd.DataFrame(features)
labels_df = pd.DataFrame(labels, columns=['filename'])


0


In [5]:
df = pd.concat([labels_df,features_df], axis=1)
df.to_csv('web_image_features.csv', index=False)
df

,filename,0,1,2,3,4,5,6,7,8,...,4598,4599,4600,4601,4602,4603,4604,4605,4606,4607
0,web_images/gulupa-100x100.jpg,-0.000000,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.000000,-0.0,...,0.801878,-0.000000,-0.000000,-0.000000,0.044111,0.486827,-0.0,-0.000000,0.358631,-0.0
1,web_images/pears.jpg,0.600314,-0.0,-0.000000,-0.0,0.347703,-0.0,-0.0,-0.000000,-0.0,...,1.063125,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,-0.000000,-0.000000,-0.0
2,web_images/strawberries1.jpg,0.934146,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.0,0.298173,-0.0,...,1.177187,-0.000000,0.376689,-0.000000,1.494727,0.355100,-0.0,-0.000000,0.128549,-0.0
3,web_images/E49BE62E-6943-E411-B834-22000AF88B16.jpg,-0.000000,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.000000,-0.0,...,1.567688,0.308267,0.182069,-0.000000,0.007600,0.801588,-0.0,-0.000000,1.020865,-0.0
4,web_images/F8CFCFB3-D645-E411-B834-22000AF88B16.jpg,-0.000000,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.000000,-0.0,...,1.226627,0.107680,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,-0.000000,0.631957,-0.0
5,web_images/A2657DDD-4643-E411-B834-22000AF88B16.jpg,-0.000000,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.000000,-0.0,...,1.133263,-0.000000,-0.000000,-0.000000,0.092329,0.529202,-0.0,-0.000000,0.331952,-0.0
6,web_images/Fruit-snacking-Future-trends-and-challenges_small.jpg,-0.000000,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.000000,-0.0,...,1.549063,-0.000000,-0.000000,-0.000000,-0.000000,0.078356,-0.0,-0.000000,0.399686,-0.0
7,web_images/Boone+Olive+Oil+CO.+Whole+Fruit+Lemon+Olive+Oil.jpg,0.325420,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.000000,-0.0,...,1.404566,0.063839,-0.000000,-0.000000,0.019060,0.226116,-0.0,-0.000000,0.304943,-0.0
8,web_images/apple-100px.jpg,0.113266,-0.0,0.083819,-0.0,0.165615,-0.0,-0.0,0.229364,-0.0,...,0.182121,-0.000000,1.044439,0.214967,-0.000000,-0.000000,-0.0,-0.000000,0.388770,-0.0
9,web_images/home-basics-fruit-baskets-fb00459-c3_100.jpg,-0.000000,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.000000,-0.0,...,0.386683,0.172640,0.192677,-0.000000,-0.000000,0.116755,-0.0,0.185478,0.378917,-0.0
